[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/vicente-gonzalez-ruiz/scalar_quantization/blob/master/docs/LloydMax_quantization.ipynb)

[![Colab](https://badgen.net/badge/Launch/on%20Google%20Colab/blue?icon=notebook)](https://colab.research.google.com/github/vicente-gonzalez-ruiz/scalar_quantization/blob/master/docs/LloydMax_quantization.ipynb)

# Lloyd-Max quantization

* Minimizes the [MSE](https://en.wikipedia.org/wiki/Mean_squared_error) of the quantization error, i.e., the expectation of the power of the quantization error
\begin{equation}
 D = \text{E}[(\mathbf{x}-\mathbf{y})^2],
\end{equation}
where $D$ is the distortion generated by the quantizer, $\mathbf{x}$ is the original signal, and $\mathbf{y}$ is the reconstructed signal.
* The [PDF](https://en.wikipedia.org/wiki/Probability_density_function) (in the analog case) or the [histogram](https://en.wikipedia.org/wiki/Histogram) (digital signals) is required. The density of quantization bins is higher in those parts of the input dynamic range where the probability of the samples is also higher.
* The quantizer must determine the decision levels, and the representation levels, which are the centroid of each bin.

## Adaptive quantization using the PDF
In the continuous case, if $M$ is the number of bins, the distortion can be expressed by
\begin{equation}
D = \sum_{i=1}^{M}\int_{\mathbf{b}_{i-1}}^{\mathbf{b}_i}(\mathbf{x}-\mathbf{c}_i)^2P(x)dx,
\end{equation}
where $\mathbf{b}_i$ is the upper decision level of the $i$-th bin, $\mathbf{c}_i$ is the representation level for the $i$-th bin, and $P(x)=f_\mathbf{x}(x)$ is the probability of finding $x$ in the signal (considered as a random variable) $\mathbf{x}$.

To minimize $D$ we must solve
\begin{equation}
\frac{\partial D}{\partial \mathbf{c}_i} = 0 = -\sum_{i=1}^M\int_{\mathbf{b}_{i-1}}^{\mathbf{b}_i}2(\mathbf{x}-\mathbf{c}_i)^2P(x)dx
\end{equation}
which boilds down to
\begin{equation}
= -\int_{\mathbf{b}_{i-1}}^{\mathbf{b}_i}2(\mathbf{x}-\mathbf{c}_i)^2P(x)dx
\end{equation}
because $\mathbf{c}_i$ is only used in one of the bins. We continue and therefore
\begin{equation}
= 2\int_{\mathbf{b}_{i-1}}^{\mathbf{b}_i}\mathbf{x}P(x)dx - 2\mathbf{c}_i\int_{\mathbf{b}_{i-1}}^{\mathbf{b}_i}P(x)dx.
\end{equation}
Solving,
\begin{equation}
\mathbf{c}_i = \frac{\int_{\mathbf{b}_{i-1}}^{\mathbf{b}_i}\mathbf{x}P(x)dx}{\int_{\mathbf{b}_{i-1}}^{\mathbf{b}_i}P(x)dx},\tag{1}
\end{equation}
i.e., the representation level $\mathbf{c}_i$ for each bin is the centroid of the probability mass in that bin. Notice that, in order to avoid a division by 0, at least one one sample must belong to each bin.

Unfortunately, such equation express that, to find the representation levels $\mathbf{c}_i$, we must determine first the decision levels $\mathbf{b}_i$. For computing them, we can now minimize $D$ respect to $\mathbf{b}_i$:
\begin{equation}
\frac{\partial D}{\partial \mathbf{b}_i} = 0,
\end{equation}
which, supposing that the bins are small enough to consider that the probability of the values of $\mathbf{x}$ is constant inside of each bin, ends up in that:
\begin{equation}
\mathbf{b}_i = \frac{\mathbf{c}_i+\mathbf{c}_{i+1}}{2},\tag{2}
\end{equation}
a result quite logical under such supposition.

## Computation of the representation levels.

Unfortunately, Equations (1) and (2) are mutually dependent. However, they can be used to compute $\{\mathbf{y}_k\}_{k=1}^M$ and $\{\mathbf{b}_k\}_{k=0}^M$ using the following iterative algorithm:

1. Initialize $\mathbf{c}_k$ /* centroids */ at random.
2. Let $\mathbf{previous\_b}=\{\mathbf{previous\_b}_k\}_{k=0}^M=0$ /* boundaries */.
2. While not reached some stopping criteria:
    1. $\mathbf{previous\_b}\leftarrow \mathbf{b}$.
    1. Compute the boundary (decision) levels $\mathbf{b}$ using Eq. (2).
    2. Update the centroids (representation levels) $\mathbf{c}$ using Eq. (1).

In [ ]:
import numpy as np
from scipy.ndimage import uniform_filter1d
# from scipy.ndimage import center_of_mass

In [ ]:
def compute_boundaries(c):
    b = uniform_filter1d(c, size=2, origin=-1)[:-1]
    b = np.concatenate(([0],b,[256]))
    print('b', b)
    return b

In [ ]:
def compute_centroids(b, P, M):
    ended = False
    c = []
    bin_size = P.size//M
    print("bin_size", bin_size)
    for i in range(len(b) - 1):
        b_i = int(round(b[i]))    #i*bin_size
        b_i_1 = int(round(b[i+1]))#(i+1)*bin_size
        if b_i == b_i_1:
            ended = True
            break
        print("b_i", b_i, "b_i_1", b_i_1)
        # See from scipy.ndimage import center_of_mass
        mass = np.sum([j*P[j] for j in range(b_i, b_i_1)])
        total_counts_in_bin = np.sum([P[j] for j in range(b_i, b_i_1)])
        assert total_counts_in_bin > 0, f"bin [{b_i}, {b_i_1}] is not used (b={b})"
        centroid = mass/total_counts_in_bin
        c.append(centroid)
    return np.array(c), ended

In [ ]:
def compute_levels(P, M, max_iters):
    total_count = np.sum(P)
    bin_count = total_count/M
    initial_boundaries = [0.]
    acc = 0
    counter = 0
    for p in P:
        acc += p
        counter += 1
        if acc > bin_count:
            initial_boundaries.append(float(counter))
            acc = 0
    initial_boundaries.append(256.)
    initial_boundaries = np.array(initial_boundaries)
    initial_centroids = 0.5 * (initial_boundaries[1:] + initial_boundaries[:-1])
    print("initial_centroids", initial_centroids, len(initial_centroids))
    c = initial_centroids
    b = initial_boundaries
    prev_b = np.zeros(b.size)
    for j in range(max_iters):
        prev_b[:] = b
        b = compute_boundaries(c)
        max_abs_error = np.max(np.abs(prev_b-b))
        print("max_abs_error", max_abs_error)
        prev_c = c
        c, ended = compute_centroids(b, P, M)
        if ended:
            break
    return b, prev_c

### Build a quantizer

In [ ]:
P = np.ones(256) # Counts for uniform distribution
#P = np.random.randint(low=0, high=2000, size=256) # Counts for random distribution
max_iters = 100
M = 2
boundaries, centroids = compute_levels(P, M, max_iters)
print('boundaries', boundaries)
print('centroids', centroids)

## Quantize an array that follows an uniform distribution

### Define the data to quantize

In [ ]:
x = np.linspace(0, 255, 256)
x

In [ ]:
try:
    import matplotlib.pyplot as plt
except:
    !pip install matplotlib
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.axes as ax
    #plt.rcParams['text.usetex'] = True
    #plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
%matplotlib inline

In [ ]:
plt.figure()
plt.title("x")
plt.xlabel("Intensity")
plt.ylabel("Value")
plt.plot(x)

### Compute the histogram(x)

In [ ]:
histogram_x, bin_edges_x = np.histogram(x, bins=256, range=(0, 256))
histogram_x

In [ ]:
plt.figure()
plt.title("Histogram(x)")
plt.xlabel("Intensity")
plt.ylabel("Count")
plt.plot(bin_edges_x[0:-1], histogram_x)

### Encode(x)

In [ ]:
k = np.searchsorted(boundaries, x, side="right") - 1
k

In [ ]:
plt.figure()
plt.title("k")
plt.xlabel("index")
plt.ylabel("value")
plt.plot(k)

### Decode(k)

In [ ]:
y = centroids[k]
y

In [ ]:
plt.figure()
plt.title("y")
plt.xlabel("decoded")
plt.ylabel("value")
plt.plot(y)

## Quantize an image

In [ ]:
try:
    from skimage import io
    from skimage.color import rgb2gray
except:
    !pip install scikit-image
    from skimage import io
    from skimage.color import rgb2gray

In [ ]:
fn = "http://www.hpca.ual.es/~vruiz/images/lena.png"

### Read the image

In [ ]:
img = io.imread(fn)
img = (rgb2gray(img)*256).astype(np.uint8)
plt.figure()
plt.title(fn)
io.imshow(img)
plt.show()

### Histogram of the image

In [ ]:
histogram_img, bin_edges_img = np.histogram(img, bins=256, range=(0, 256))
histogram_img[histogram_img==0] = 1
print("histogram", histogram_img)
print("\bin_edges", bin_edges_img)
print(len(histogram_img))

In [ ]:
plt.figure()
plt.title("Histogram")
plt.xlabel("Intensity")
plt.ylabel("Count")
plt.plot(bin_edges_img[0:-1], histogram_img)

### Build the quantizer

In [ ]:
P = histogram_img
M = 128
boundaries_img, centroids_img = compute_levels(P, M, max_iters=100)

In [ ]:
print(boundaries_img, len(boundaries_img))

In [ ]:
print(centroids_img, len(centroids_img))

In [ ]:
print(histogram_img)

### Encode

In [ ]:
indexes_img = np.searchsorted(boundaries_img, img) - 1

In [ ]:
print(indexes_img.shape)
print(np.min(indexes_img))
print(np.max(indexes_img))
print(np.unique(indexes_img))
print(len(np.unique(indexes_img)))

In [ ]:
plt.figure()
plt.title('k')
io.imshow(indexes_img, cmap="gray")
plt.show()

### Decode

In [ ]:
decoded_img = centroids_img[indexes_img]#.astype(np.uint8)

In [ ]:
print(decoded_img.min(), decoded_img.max())

In [ ]:
plt.figure()
plt.title("decoded")
io.imshow(decoded_img, cmap="gray")
plt.show()

## Quantization function

In [ ]:
x = np.linspace(0, 255, 256) # Input samples
x

In [ ]:
indexes_x = np.searchsorted(boundaries_img, x) - 1
indexes_x

In [ ]:
decoded_x = centroids_img[indexes_x]#.astype(np.uint8)
decoded_x

In [ ]:
xlabel = "Input Sample"
ylabel = "Reconstructed Sample"
title = f"Lloyd-Max Quantizer ({fn}, $\M={M}$)"

ax1 = plt.subplot()
counts, bins = np.histogram(img, range(256))
l1 = ax1.bar(bins[:-1] - 0.5, counts, width=1, edgecolor='none')
ax2 = ax1.twinx()
l2, = ax2.plot(x[1:], decoded_x[1:], color='m')

plt.title("Histogram VS Quantization Function")
plt.legend([l1, l2], ["Histogram", "Lloyd-Max Quantizer"])
ax1.yaxis.set_label_text("Pixel Value Count")
ax2.yaxis.set_label_text("Reconstructed Value")
ax1.xaxis.set_label_text("Input Sample")
plt.show()

Notice that in those input ranges where the number of gray-tones are more frequent, the resolution of the quantizer is increased, and the representation levels are placed where the MSE is minimized.

## Testing the library

In [ ]:
try:
    from scalar_quantization.LloydMax_quantization import LloydMax_Quantizer as Quantizer                          
    from scalar_quantization.LloydMax_quantization import name as quantizer_name
except:
    !pip install "scalar_quantization @ git+https://github.com/vicente-gonzalez-ruiz/scalar_quantization"
    from scalar_quantization.LloydMax_quantization import LloydMax_Quantizer as Quantizer
    from scalar_quantization.LloydMax_quantization import name as quantizer_name

In [ ]:
Q_step = 128
Q = Quantizer(Q_step=Q_step, counts=histogram_img)
print("decision_levels =", Q.get_decision_levels())
print("representation_levels =", Q.get_representation_levels())

In [ ]:
#quantized_img, indexes = Q.encode_and_decode(img.flatten())
quantized_img, indexes = Q.encode_and_decode(img)

In [ ]:
#gray_image.show_normalized(indexes.reshape(img.shape), fn + "000.png")
#gray_image.show(quantized_img, "y")
#gray_image.show(quantized_img, "y")

In [ ]:
plt.figure()
plt.title('y')
io.imshow(quantized_img, cmap="gray")
plt.show()

In [ ]:
Q_step = 32
Q = Quantizer(Q_step=Q_step, counts=histogram_img)
print("decision_levels =", Q.get_decision_levels())
print("representation_levels =", Q.get_representation_levels())
quantized_img, indexes = Q.encode_and_decode(img)
plt.figure()
plt.title('y')
io.imshow(quantized_img, cmap="gray")
plt.show()

In [ ]:
Q_step = 2
Q = Quantizer(Q_step=Q_step, counts=histogram_img)
print("decision_levels =", Q.get_decision_levels())
print("representation_levels =", Q.get_representation_levels())
quantized_img, indexes = Q.encode_and_decode(img)
plt.figure()
plt.title('y')
io.imshow(quantized_img, cmap="gray")
plt.show()

In [ ]:
input()